In [1]:
import json
from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
# Ignore warnings
import warnings

In [24]:
# pd.set_option('display.max_rows', None)

#### Helpfer functions

In [137]:
def list_json_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]
    for subdir in subdirs:
        files = os.walk(subdir).__next__()[2]
        if (len(files) > 0):                                                                                          
            for file in files:
                if file.endswith('.json'):
                    r.append(os.path.join(subdir, file))                   
    return r

def leaf_dirs(dir):
    max_depth = 0
    leaf_dir = []
    for dirpath, dirnames, filenames in os.walk(dir):
        depth= len(dirpath.split(os.sep))
        if max_depth < depth:
            max_depth= depth
            leaf_dir = [dirpath]
        elif max_depth == depth:
            leaf_dir.append(dirpath)
    return(leaf_dir)

def offset(file_list):
    res1=[]
    res2=[]
    for json_file in file_list:
        result=[]
        length=[]
        data=None
        f=open(json_file)
        data=json.load(f)
        heap_start=data['HEAP_START']
        keys=[None,None,None,None,None,None]
        length=[data['KEY_A_LEN'],data['KEY_B_LEN'],data['KEY_C_LEN'],data['KEY_D_LEN'],data['KEY_E_LEN'],data['KEY_F_LEN']]
    
        try:
            if int(data['KEY_A_LEN']) > 0:
                keys[0]=data['KEY_A_ADDR']
            else:
                pass      
        except :
            pass
    
        try:
            if int(data['KEY_B_LEN']) > 0:
                keys[1]=data['KEY_B_ADDR']
            else:
                pass      
        except :
            pass
    
        try:
            if int(data['KEY_C_LEN']) > 0:
                keys[2]=data['KEY_C_ADDR']
            else:
                pass      
        except :
            pass
    
        try:
            if int(data['KEY_D_LEN']) > 0:
                keys[3]=data['KEY_D_ADDR']
            else:
                pass      
        except :
            pass
    
        try:
            if int(data['KEY_E_LEN']) > 0:
                keys[4]=data['KEY_E_ADDR']
            else:
                pass      
        except :
            pass


        try:
            if int(data['KEY_F_LEN']) > 0:
                keys[5]=data['KEY_F_ADDR']
            else:
                pass      
        except :
            pass


        for e in keys:
            if e:
                result.append(hex(int(e, 16)-int(heap_start, 16)))
            else:
                result.append(None)
        res1.append(result)
        res2.append(length)
    return res1, res2

def list_txt_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]
    for subdir in subdirs:
        files = os.walk(subdir).__next__()[2]
        if (len(files) > 0):                                                                                          
            for file in files:               
                if file.endswith('.txt'):
                    r.append(os.path.join(subdir, file))                   
    return r

def create_Dataframe(file,offsets,lengths,N):
    columns=['Data']
    df=pd.read_csv(file,delimiter='\t',names=columns,header=None)
    df['offset']=df['Data'].str[:8]
    df['hex_values']=df['Data'].str[10:]
    df['hex_values']=df['hex_values'].str[:-10]
    df=df.drop('Data',axis=1)
    df=df[df['offset'] !='*']
    val=df['offset']
    value_list=val.tolist()
    classes, key_offsets, key_start=create_labels(file,offsets,lengths,value_list)
    df['class']= classes
    df.index = np.arange(1, len(df) + 1)
    all_data=block100_set(df,key_offsets,key_start,N)
    return df, all_data


def create_labels(file,offset,length,addresses):

    classes=[0]*len(addresses)
    temp_offset,key_offsets,clean_len= clean_offset_len(offset,length)
    for e in range(len(temp_offset)):
        if temp_offset[e]:
            id=addresses.index(temp_offset[e])
            for k in range(clean_len[e]):
                classes[id+k] = 1
    return classes, key_offsets, temp_offset


def clean_offset_len(offset,length):
    key_offsets=[]
    key_lengths=[0]*len(length)
    clean_offsets=[None]*len(offset)

    for j in range(len(length)):       
        key_lengths[j] = int(int(length[j])/8)
        
        
    for i in range(len(offset)):
        if offset[i]:
            e=offset[i]
            
            for k in range(0,key_lengths[i]):
                key_offsets.append(e)
                e=hex(int(e, 16) + int('8', 16))
                
    for i in range(len(offset)):
        if offset[i]:
            temp=offset[i].split('x')
            comp='0'*(8-len(temp[1]))
            clean_offsets[i]=comp+temp[1]
  
    return clean_offsets, key_offsets, key_lengths
                


def version_df(txt_files,json_files,N):
    dataframes=[]
    i=0
    txt_files.sort()
    json_files.sort()    
    offsets, lengths= offset(json_files)
    for file in txt_files:
        df, block_df=create_Dataframe(file,offsets[i],lengths[i],N)
        i=i+1
        dataframes.append(block_df)
    all_data=pd.concat(dataframes)
    return all_data

def hex_transform(e):
    return hex(int(str(e) , 16))

def block100_set(df,key_offsets,key_start_list,N):
    
    offset_start=''
    offset_end=''
    i=1
    temp=[]
    label=0
    key_start=None
    dataframes=[]
    key_position=None
    hot_code=[0]*N
    for index, row in df.iterrows():
        
        if i == 1:
            offset_start = row['offset']
        elif i == N:
            offset_end = row['offset']
            
        if i<N and len(dataframes) < df.shape[0] // N:
            temp.append(row["hex_values"])
            label = int(label)+ int(row["class"])

            if hex_transform(row['offset']) in key_offsets:                    
                hot_code[i-1]=1
            if row['offset'] in key_start_list:
                key_start= str(row['offset'])
                key_position=i-1
                
            i=i+1
            
        elif len(dataframes) == df.shape[0] // N:
            if i == 1:
                offset_start = row['offset']
            
            temp.append(row['hex_values'])
            label = int(label)+ int(row["class"])
            if hex_transform(row['offset']) in key_offsets:
                hot_code[i-1]=1
            if row['offset'] in key_start_list:
                key_start= str(row['offset'])
                key_position=i-1
                
            i=i+1
            
        elif i == N and len(dataframes) < df.shape[0] // N:
            
            temp.append(row['hex_values'])
            hex_val=' '.join(temp)

            label = int(label)+ int(row["class"])
            if hex_transform(row['offset']) in key_offsets:
                hot_code[i-1]=1
            if row['offset'] in key_start_list:
                key_start= str(row['offset'])
                key_position=i-1
            
            temp_df = pd.DataFrame(columns=['offset_range','hex_values','class','key_start_offset','key_start_position','elements_classes'])
            temp_df.loc["0",'key_start_offset']=key_start
            temp_df.loc["0",'hex_values']=hex_val
            temp_df.loc["0",'key_start_position']=key_position
            temp_df.loc["0",'elements_classes']=hot_code
            temp_df.loc["0","offset_range"]=str(offset_start)+"-"+str(offset_end)
            key_start=None
            key_position = None
            if label >0:
                temp_df.loc["0",'class']=1
            else:
                temp_df.loc["0",'class']=0
            
            label=0
            temp=[]
            hot_code=[0]*N
            dataframes.append(temp_df)
            i=1
            
    offset_end= row['offset']      
    hex_val=' '.join(temp) 
    while (len(hex_val)<(20*N)-1):
        hex_val=hex_val +" 0000 0000 0000 0000"    
    temp_df = pd.DataFrame(columns=['offset_range','hex_values','class','key_start_offset','key_start_position','elements_classes'])
    temp_df.loc["0",'key_start_offset']=key_start
    temp_df.loc["0",'hex_values']=hex_val
    temp_df.loc["0",'key_start_position']=key_position
    temp_df.loc["0",'elements_classes']=hot_code
    temp_df.loc["0","offset_range"]=str(offset_start)+"-"+str(offset_end)
    key_start=None
    key_position=None
        
    if label >0:
        temp_df.loc["0",'class']=1
    else:
        temp_df.loc["0",'class']=0        
    dataframes.append(temp_df)
    all_data=pd.concat(dataframes,ignore_index=True)

    
    return all_data

def version_save_set(data,direc):
    direc=direc.replace('\\', '/')
    dir_list=direc.split('/')
    folder=dir_list[2]
    dir_list=dir_list[3:]
    name='-'.join(dir_list)
    name='./CSV_Data/'+folder+'/'+name+'.csv'
    data.to_csv(name,sep='\t',index=False)
    return

def file_save_set(data,file):
        filename=file[:-9]
        filename=filename.replace('\\', '/')
        filename_list=filename.split('/')        
        folder1=filename_list[2]
        folder2=filename_list[6]
        filename_list1=filename_list[3:-1]
        name='-'.join(filename_list1)       
        name_dir='./CSV_Data/'+folder1+'/'+name
        name_file='/'+folder2+'.csv'
        output_dir=Path(name_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
        name=name_dir+name_file
        data.to_csv(name,sep='\t',index=False)
       
        return
    


### Paths

In [149]:
custom_dir='./Data/Training/basic/V_8_8_P1/64'

In [153]:
train_dir = "./Data/Training/"
test_dir = "./Data/Performance Test/"
eval_dir = "./Data/Validation/"

### Files and directories extraction

In [154]:
# train_leaf_dir=leaf_dirs(train_dir)
# test_leaf_dir=leaf_dirs(test_dir)
validation_leaf_dir=leaf_dirs(eval_dir)

In [6]:
all_txt_files=list_txt_files(custom_dir)
all_json_files=list_json_files(custom_dir)

offsets, lengths= offset(all_json_files)

test_file=all_txt_files[0]
test_offset=offsets[0]
test_length=lengths[0]

### Creating Dataframe from 1 file

In [7]:
df, block_df=create_Dataframe(test_file,test_offset,test_length,200)

In [8]:
block_df

,offset_range,hex_values,class,key_start_offset,key_start_position,elements_classes
0,00000000-000006d0,0000 0000 0000 0000 5102 0000 0000 0000 0607 0...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,000006d8-00000de8,4100 0000 0000 0000 0000 0000 0000 0000 c100 0...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,00000df0-00001428,0200 0000 0000 0000 b116 a9d2 d87f 0000 80f7 a...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,00001430-00001c58,cd4c f7ea 0000 0000 2100 0000 0000 0000 0200 0...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,00001c60-00002298,40e6 afd2 d87f 0000 2100 0000 0000 0000 908c a...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,000022a0-000028d8,80d6 afd2 d87f 0000 2100 0000 0000 0000 d092 a...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,000028e0-00002f98,80eb afd2 d87f 0000 2100 0000 0000 0000 1099 a...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,00002fa0-000035d8,0200 0000 0000 0000 0461 a9d2 d87f 0000 4093 b...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,000035e0-00003c18,0200 0000 0000 0000 374b a9d2 d87f 0000 20f1 a...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,00003c20-00004258,0100 0000 0080 0000 e1ee a8d2 d87f 0000 ce1c a...,0,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [35]:
block_df.iloc[30]['key_position'][155:165]

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]

In [34]:
block_df.iloc[30]['hex_values'][155:163]

['eb6a 3c94 52f8 e22e',
 '0eb0 d0c5 b169 7b70',
 'd4da 9c25 76e4 32a9',
 '4ef5 9b21 527e 53c7',
 '4052 f439 647d f8db',
 '96ec 92f2 63df f102',
 '3479 e6b2 0a3f bc01',
 '876a 2997 078d c2ac']

In [33]:
len(block_df.iloc[30]['hex_values'])

200

### Creating Dataset for 1 version (1059 files)

In [ ]:
data = version_df(all_txt_files,all_json_files,200)

### Preparing directories

In [146]:
leaf_dir=[]
leaf_dir.extend(train_leaf_dir)
leaf_dir.extend(test_leaf_dir)
leaf_dir.extend(validation_leaf_dir)

### Saving version dataframes in CSV

In [158]:
%%time
for direc in validation_leaf_dir:
    txt_files=list_txt_files(direc)
    json_files=list_json_files(direc)
    dataset=version_df(txt_files,json_files,200)
    version_save_set(dataset,direc)

CPU times: user 1h 30min 51s, sys: 8.26 s, total: 1h 30min 59s
Wall time: 1h 33min 18s


### Saving file dataframes in CSV

In [ ]:
for direc in test_dir:
    txt_files=list_txt_files(direc)
    json_files=list_json_files(direc)
    offsets, lengths= offset(json_files)
    for e in range(len(txt_files)):
        file=txt_files[e]
        df,data=create_Dataframe(file,offsets[e],lengths[e],200)
        file_save_set(data,file)